In [1]:
from pathlib import Path
import os 

In [2]:
file_path = Path(os.getcwd()) / "data" / "raw" / "sample.txt"
print(file_path)

/Users/mahmoudmohamed/Library/Mobile Documents/com~apple~CloudDocs/Project/ultimate_rag/p1-naive-rag-run1/RAG QA System Application/notebooks/data/raw/sample.txt


In [3]:
base_dir = os.makedirs(Path(os.getcwd()) / "data" / "raw", exist_ok=True)

In [4]:
if file_path.is_file():
    print("File exists.")
else:
    with open(file_path, 'w') as f:
        f.write("This is a sample file.")

File exists.


In [5]:
import logging 

In [6]:
log = logging.getLogger()

In [7]:
log.setLevel(logging.INFO)

In [8]:
getattr(log, 'handlers')

[]

In [12]:
print(log.__getattribute__('level'))

20


In [13]:
log.__dict__

{'filters': [],
 'name': 'root',
 'level': 20,
 'parent': None,
 'propagate': True,
 'handlers': [],
 'disabled': False,
 '_cache': {}}

In [14]:
log.addHandler(logging.StreamHandler())
log.addHandler(logging.FileHandler('app.log'))

In [16]:
getattr(log,'handlers',log.handlers)

[<StreamHandler stderr (NOTSET)>,
 <FileHandler /Users/mahmoudmohamed/Library/Mobile Documents/com~apple~CloudDocs/Project/ultimate_rag/p1-naive-rag-run1/RAG QA System Application/notebooks/app.log (NOTSET)>]

In [18]:
for handler in log.handlers:
    print(handler)
    
# if i wanted to remove all handlers 
for handler in log.handlers[:]:
    log.removeHandler(handler)
    
# why i need to write it log.handlers[:] is because if i iterate directly over log.handlers and remove handlers while iterating it will cause issues as the list is being modified during iteration. By iterating over a shallow copy of the list (using log.handlers[:]), I can safely remove handlers from the original list without affecting the iteration process.

<StreamHandler stderr (NOTSET)>
<FileHandler /Users/mahmoudmohamed/Library/Mobile Documents/com~apple~CloudDocs/Project/ultimate_rag/p1-naive-rag-run1/RAG QA System Application/notebooks/app.log (NOTSET)>


In [21]:
log.addHandler(logging.StreamHandler())
log.addHandler(logging.FileHandler('app.log'))
log.addHandler(logging.NullHandler())
for handler in log.handlers:
    log.removeHandler(handler)

log.handlers
# so why this code worked and handers were removed ? Because at the time of this loop, log.handlers is already empty (since we removed all handlers in the previous loop). Therefore, there are no handlers left to iterate over, and the loop effectively does nothing.
# why the iteration skipped filehandler  and removed stream and null handlers? because when we remove a handler from log.handlers, the list is modified in place. This means that the indices of the remaining handlers shift down by one position. As a result, when the loop moves to the next index, it skips over the handler that shifted into the current index position after the removal.

[<FileHandler /Users/mahmoudmohamed/Library/Mobile Documents/com~apple~CloudDocs/Project/ultimate_rag/p1-naive-rag-run1/RAG QA System Application/notebooks/app.log (NOTSET)>]

In [25]:
# why the iteration skipped filehandler  and removed stream and null handlers? because when we remove a handler from log.handlers, the list is modified in place. This means that the indices of the remaining handlers shift down by one position. As a result, when the loop moves to the next index, it skips over the handler that shifted into the current index position after the removal.
#another numerical example to illustrate this behavior:
# suppose we have a list of handlers represented by their indices: [0, 1, 2]
# we start iterating over the list:
# - first iteration: index 0 (handler 0) -> we remove handler 0
# - the list now looks like this: [1, 2] (handler 1 is now at index 0, handler 2 is at index 1)
# - second iteration: index 1 (handler 2) -> we remove handler 2
# - the list now looks like this: [1]
# - the iteration ends because there are no more indices to process.
# as a result, handler 1 (originally at index 1) was never processed because it shifted to index 0 after handler 0 was removed, and the loop moved on

lst = ['a', 'b', 'c']
for i in range(len(lst)):
    lst.remove(lst[i])
print(lst)  # Output will be ['b', 'c'] because 'a' was


IndexError: list index out of range

In [26]:
# Visual demonstration of the index-shifting phenomenon
lst = ['a', 'b', 'c']
print(f"Initial list: {lst}")
print(f"Loop will iterate: range({len(lst)}) = {list(range(len(lst)))}")
print("-" * 60)

# Create a copy to iterate with indices
lst_demo = ['a', 'b', 'c']
iteration_count = len(lst_demo)

for i in range(iteration_count):
    print(f"\nIteration {i + 1} (i={i}):")
    print(f"  Current list: {lst_demo}")
    print(f"  Trying to access index {i}")
    
    if i < len(lst_demo):
        element = lst_demo[i]
        print(f"  Element at index {i}: '{element}'")
        lst_demo.remove(element)
        print(f"  Removed '{element}'")
        print(f"  List after removal: {lst_demo}")
    else:
        print(f"  ❌ ERROR: Index {i} is out of range!")
        print(f"  List only has {len(lst_demo)} element(s), max index is {len(lst_demo) - 1}")
        break

print("\n" + "=" * 60)
print(f"Final result: {lst_demo}")
print(f"Elements that were NEVER processed: {[item for item in ['a', 'b', 'c'] if item in lst_demo]}")

Initial list: ['a', 'b', 'c']
Loop will iterate: range(3) = [0, 1, 2]
------------------------------------------------------------

Iteration 1 (i=0):
  Current list: ['a', 'b', 'c']
  Trying to access index 0
  Element at index 0: 'a'
  Removed 'a'
  List after removal: ['b', 'c']

Iteration 2 (i=1):
  Current list: ['b', 'c']
  Trying to access index 1
  Element at index 1: 'c'
  Removed 'c'
  List after removal: ['b']

Iteration 3 (i=2):
  Current list: ['b']
  Trying to access index 2
  ❌ ERROR: Index 2 is out of range!
  List only has 1 element(s), max index is 0

Final result: ['b']
Elements that were NEVER processed: ['b']


In [27]:
# Demonstration: Iterator behavior when modifying list during iteration

print("=" * 70)
print("SCENARIO 1: for item in lst (WRONG - skips elements)")
print("=" * 70)

lst = ['StreamHandler', 'FileHandler', 'NullHandler']
print(f"Initial list: {lst}\n")

# Simulating what Python's iterator does internally
iteration = 0
for item in lst:
    print(f"Iteration {iteration + 1}:")
    print(f"  Iterator fetches: '{item}' (from current position)")
    print(f"  Current list state: {lst}")
    print(f"  Removing: '{item}'")
    lst.remove(item)
    print(f"  After removal: {lst}")
    print(f"  ⚠️  Iterator moves to NEXT position (doesn't go back!)\n")
    iteration += 1

print(f"Final result: {lst}")
print(f"⚠️  FileHandler was SKIPPED!\n")

print("\n" + "=" * 70)
print("SCENARIO 2: for item in lst[:] (CORRECT - copy prevents skipping)")
print("=" * 70)

lst = ['StreamHandler', 'FileHandler', 'NullHandler']
lst_copy = lst[:]  # Create a copy
print(f"Original list: {lst}")
print(f"Iterator copy: {lst_copy}")
print(f"Note: Iterator uses the COPY, removals affect ORIGINAL\n")

iteration = 0
for item in lst_copy:  # Iterate over the copy
    print(f"Iteration {iteration + 1}:")
    print(f"  Iterator fetches from COPY: '{item}'")
    print(f"  Copy state (unchanged): {lst_copy}")
    print(f"  Original list state: {lst}")
    print(f"  Removing '{item}' from ORIGINAL list")
    lst.remove(item)
    print(f"  Original after removal: {lst}")
    print(f"  Copy remains unchanged: {lst_copy}\n")
    iteration += 1

print(f"Final original: {lst}")
print(f"Final copy: {lst_copy}")
print(f"✅ All items were processed!")

print("\n" + "=" * 70)
print("KEY INSIGHT")
print("=" * 70)
print("• for item in lst: Iterator reads from lst AS IT CHANGES")
print("• When you remove an item, remaining items shift left")
print("• Iterator position still moves forward → skips shifted items")
print("• for item in lst[:]: Iterator reads from FROZEN COPY")
print("• Removals don't affect the copy → all items are seen")

SCENARIO 1: for item in lst (WRONG - skips elements)
Initial list: ['StreamHandler', 'FileHandler', 'NullHandler']

Iteration 1:
  Iterator fetches: 'StreamHandler' (from current position)
  Current list state: ['StreamHandler', 'FileHandler', 'NullHandler']
  Removing: 'StreamHandler'
  After removal: ['FileHandler', 'NullHandler']
  ⚠️  Iterator moves to NEXT position (doesn't go back!)

Iteration 2:
  Iterator fetches: 'NullHandler' (from current position)
  Current list state: ['FileHandler', 'NullHandler']
  Removing: 'NullHandler'
  After removal: ['FileHandler']
  ⚠️  Iterator moves to NEXT position (doesn't go back!)

Final result: ['FileHandler']
⚠️  FileHandler was SKIPPED!


SCENARIO 2: for item in lst[:] (CORRECT - copy prevents skipping)
Original list: ['StreamHandler', 'FileHandler', 'NullHandler']
Iterator copy: ['StreamHandler', 'FileHandler', 'NullHandler']
Note: Iterator uses the COPY, removals affect ORIGINAL

Iteration 1:
  Iterator fetches from COPY: 'StreamHandler

In [28]:
lst1 = ['a', 'b', 'c']
lst2 = lst1 

for item in lst2:
    lst2.remove(item)
print(lst2)  # Output will be ['b', 'c'] because 'a'

['b']


In [31]:
lst1 = ['a', 'b', 'c']
lst2 = lst1[:]

for item in lst2:
    lst2.remove(item)
print(lst2)  

['b']


In [32]:
lst1 = ['a', 'b', 'c']


for item in lst1.copy():
    lst1.remove(item)
print(lst1)  

[]


In [34]:
ls1 = ['a', 'b', 'c']
lst2 = ls1[:]
print(hex(id(lst2)))
print(hex(id(ls1))) 


for item in ls1:
    print(hex(id(item)))

print("="*20)
for item in lst2:
    print(hex(id(item)))



0x10cd8d580
0x1124a07c0
0x1072784b0
0x1072784e0
0x107278510
0x1072784b0
0x1072784e0
0x107278510


In [ ]:
# CLARIFICATION: Reference vs Shallow Copy

print("=" * 70)
print("1. REFERENCE (lst2 = lst1) - BOTH POINT TO SAME LIST")
print("=" * 70)

lst1 = ['a', 'b', 'c']
lst2 = lst1  # This is NOT a copy! Just another name for the same list

print(f"lst1: {lst1}")
print(f"lst2: {lst2}")
print(f"lst1 is lst2: {lst1 is lst2}")  # True - same object
print(f"id(lst1): {id(lst1)}")
print(f"id(lst2): {id(lst2)}")  # Same memory address!

print("\nRemoving 'a' from lst1...")
lst1.remove('a')

print(f"lst1: {lst1}")
print(f"lst2: {lst2}")  # lst2 also changed!
print("⚠️  Both changed because they're THE SAME list!\n")

print("=" * 70)
print("2. SHALLOW COPY (lst2 = lst1[:]) - TWO SEPARATE LISTS")
print("=" * 70)

lst1 = ['a', 'b', 'c']
lst2 = lst1[:]  # Creates a NEW list with the same elements

print(f"lst1: {lst1}")
print(f"lst2: {lst2}")
print(f"lst1 is lst2: {lst1 is lst2}")  # False - different objects
print(f"id(lst1): {id(lst1)}")
print(f"id(lst2): {id(lst2)}")  # Different memory address!

print("\nRemoving 'a' from lst1...")
lst1.remove('a')

print(f"lst1: {lst1}")
print(f"lst2: {lst2}")  # lst2 is UNCHANGED!
print("✅ lst2 is independent!\n")

print("=" * 70)
print("3. WHY IT'S CALLED 'SHALLOW' - Only matters for nested lists")
print("=" * 70)

lst1 = [['a'], ['b'], ['c']]  # List of lists
lst2 = lst1[:]  # Shallow copy

print(f"lst1: {lst1}")
print(f"lst2: {lst2}")
print(f"lst1 is lst2: {lst1 is lst2}")  # False - outer lists are different
print(f"lst1[0] is lst2[0]: {lst1[0] is lst2[0]}")  # True - inner lists are SAME!

print("\nModifying inner list lst1[0]...")
lst1[0].append('x')

print(f"lst1: {lst1}")
print(f"lst2: {lst2}")  # lst2[0] also changed!
print("⚠️  Inner lists are shared!\n")

print("But removing from outer list lst1...")
lst1.remove(['b'])

print(f"lst1: {lst1}")
print(f"lst2: {lst2}")  # lst2 unchanged
print("✅ Outer list modification doesn't affect lst2\n")

print("=" * 70)
print("KEY TAKEAWAYS")
print("=" * 70)
print("• lst2 = lst1        → Reference (same list, same address)")
print("• lst2 = lst1[:]     → Shallow copy (new list, different address)")
print("• For simple items (str, int): shallow copy is fully independent")
print("• For nested objects: outer list is copied, inner objects are shared")
print("• Use copy.deepcopy() for complete independence of nested structures")

In [36]:
import copy 

lst1=['a', 'b', 'c']
lst2=copy.deepcopy(lst1)

print(hex(id(lst2)))
print(hex(id(lst1)))

for item in lst1:
    print(hex(id(item)))

print("------------------")

for item in lst2:
    print(hex(id(item)))




0x10ce58e00
0x112483880
0x1072784b0
0x1072784e0
0x107278510
------------------
0x1072784b0
0x1072784e0
0x107278510


In [40]:
# Deep Copy - When addresses ARE the same vs DIFFERENT

import copy

print("=" * 70)
print("IMMUTABLE ELEMENTS (strings, ints) - SAME ADDRESSES")
print("=" * 70)

lst1 = ['a', 'b', 'c']
lst2 = copy.deepcopy(lst1)

print(f"lst1: {lst1}, id: {hex(id(lst1))}")
print(f"lst2: {lst2}, id: {hex(id(lst2))}")
print(f"Lists have different addresses: {id(lst1) != id(lst2)}")

print("\nElements:")
for i in range(len(lst1)):
    print(f"  lst1[{i}]='{lst1[i]}' id:{hex(id(lst1[i]))} | lst2[{i}]='{lst2[i]}' id:{hex(id(lst2[i]))} | Same? {lst1[i] is lst2[i]}")

print("\n✅ Strings have SAME addresses - Python reuses immutable objects!\n")

print("=" * 70)
print("MUTABLE ELEMENTS (nested lists) - DIFFERENT ADDRESSES")
print("=" * 70)

lst1 = [['a', 'b'], ['c', 'd']]
lst2_shallow = lst1[:]
lst3_deep = copy.deepcopy(lst1)

print("SHALLOW COPY:")
print(f"  lst1:         {lst1}, id: {hex(id(lst1))}")
print(f"  lst2_shallow: {lst2_shallow}, id: {hex(id(lst2_shallow))}")
print(f"  Outer lists different? {id(lst1) != id(lst2_shallow)}")
print(f"\n  Inner LISTS (the list objects themselves):")
for i in range(len(lst1)):
    print(f"    lst1[{i}] id:{hex(id(lst1[i]))} | lst2_shallow[{i}] id:{hex(id(lst2_shallow[i]))} | Same? {lst1[i] is lst2_shallow[i]}")
print("  ⚠️  Inner list OBJECTS are SHARED!\n")

print("DEEP COPY:")
print(f"  lst1:      {lst1}, id: {hex(id(lst1))}")
print(f"  lst3_deep: {lst3_deep}, id: {hex(id(lst3_deep))}")
print(f"  Outer lists different? {id(lst1) != id(lst3_deep)}")
print(f"\n  Inner LISTS (the list objects themselves):")
for i in range(len(lst1)):
    print(f"    lst1[{i}] id:{hex(id(lst1[i]))} | lst3_deep[{i}] id:{hex(id(lst3_deep[i]))} | Same? {lst1[i] is lst3_deep[i]}")
print("  ✅ Inner list OBJECTS are INDEPENDENT!\n")

print("  But the STRINGS inside those lists:")
for i in range(len(lst1)):
    for j in range(len(lst1[i])):
        print(f"    lst1[{i}][{j}]='{lst1[i][j]}' id:{hex(id(lst1[i][j]))} | lst3_deep[{i}][{j}]='{lst3_deep[i][j]}' id:{hex(id(lst3_deep[i][j]))} | Same? {lst1[i][j] is lst3_deep[i][j]}")
print("  ⚠️  String ELEMENTS are still SHARED (immutable optimization)!\n")

print("=" * 70)
print("KEY INSIGHT - Three Levels of Memory")
print("=" * 70)
print("For lst1 = [['a', 'b'], ['c', 'd']]:")
print("\nLevel 1 - Outer list container:")
print("  • Shallow copy: DIFFERENT address")
print("  • Deep copy:    DIFFERENT address")
print("\nLevel 2 - Inner list objects:")
print("  • Shallow copy: SAME address (shared!)")
print("  • Deep copy:    DIFFERENT address (independent!)")
print("\nLevel 3 - String elements:")
print("  • Shallow copy: SAME address (immutable)")
print("  • Deep copy:    SAME address (immutable - Python optimization)")
print("\n✅ Deep copy creates new CONTAINERS but reuses IMMUTABLE elements")

IMMUTABLE ELEMENTS (strings, ints) - SAME ADDRESSES
lst1: ['a', 'b', 'c'], id: 0x1124fdf00
lst2: ['a', 'b', 'c'], id: 0x1124ff5c0
Lists have different addresses: True

Elements:
  lst1[0]='a' id:0x1072784b0 | lst2[0]='a' id:0x1072784b0 | Same? True
  lst1[1]='b' id:0x1072784e0 | lst2[1]='b' id:0x1072784e0 | Same? True
  lst1[2]='c' id:0x107278510 | lst2[2]='c' id:0x107278510 | Same? True

✅ Strings have SAME addresses - Python reuses immutable objects!

MUTABLE ELEMENTS (nested lists) - DIFFERENT ADDRESSES
SHALLOW COPY:
  lst1:         [['a', 'b'], ['c', 'd']], id: 0x1124fd4c0
  lst2_shallow: [['a', 'b'], ['c', 'd']], id: 0x10cd8b4c0
  Outer lists different? True

  Inner LISTS (the list objects themselves):
    lst1[0] id:0x1124fd280 | lst2_shallow[0] id:0x1124fd280 | Same? True
    lst1[1] id:0x10cd893c0 | lst2_shallow[1] id:0x10cd893c0 | Same? True
  ⚠️  Inner list OBJECTS are SHARED!

DEEP COPY:
  lst1:      [['a', 'b'], ['c', 'd']], id: 0x1124fd4c0
  lst3_deep: [['a', 'b'], ['c', 

In [41]:
lst1 = [['a', 'b'], ['c', 'd']]
lst2 = copy.deepcopy(lst1)

lst1[0].append('x')

In [42]:
lst1 , lst2

([['a', 'b', 'x'], ['c', 'd']], [['a', 'b'], ['c', 'd']])

In [44]:
lst1 = [['a', 'b'], ['c', 'd']]
lst2 = copy.copy(lst1)
print(hex(id(lst1)), hex(id(lst2)))
lst1[0].append('x')

lst1 , lst2 

0x112513c00 0x11243d840


([['a', 'b', 'x'], ['c', 'd']], [['a', 'b', 'x'], ['c', 'd']])

In [ ]:
# WHY Shallow Copy Behaves This Way - THE KEY INSIGHT

import copy

lst1 = [['a', 'b'], ['c', 'd']]
lst2 = copy.copy(lst1)  # Shallow copy

print("=" * 70)
print("UNDERSTANDING SHALLOW COPY BEHAVIOR")
print("=" * 70)

print("\nOuter list addresses (the containers):")
print(f"lst1: {hex(id(lst1))}")
print(f"lst2: {hex(id(lst2))}")
print(f"Different? {id(lst1) != id(lst2)} ✅")

print("\n" + "-" * 70)
print("CRITICAL: Inner list addresses (what they POINT TO):")
print("-" * 70)
print(f"lst1[0]: {hex(id(lst1[0]))} → points to {lst1[0]}")
print(f"lst2[0]: {hex(id(lst2[0]))} → points to {lst2[0]}")
print(f"Same object? {lst1[0] is lst2[0]} ⚠️  SHARED!")

print(f"\nlst1[1]: {hex(id(lst1[1]))} → points to {lst1[1]}")
print(f"lst2[1]: {hex(id(lst2[1]))} → points to {lst2[1]}")
print(f"Same object? {lst1[1] is lst2[1]} ⚠️  SHARED!")

print("\n" + "=" * 70)
print("WHAT HAPPENS WHEN YOU MODIFY")
print("=" * 70)

print(f"\nBefore: lst1 = {lst1}, lst2 = {lst2}")
print("\nExecuting: lst1[0].append('x')")
print("  ↓")
print("  You're NOT modifying lst1 itself")
print("  You're modifying the INNER LIST that lst1[0] points to")
print("  ↓")
print("  Since lst2[0] points to the SAME inner list...")
print("  ↓")
print("  Both see the change!")

lst1[0].append('x')

print(f"\nAfter:  lst1 = {lst1}")
print(f"        lst2 = {lst2}")
print(f"\n✅ Both changed because lst1[0] and lst2[0] point to the same list!\n")

print("=" * 70)
print("VISUAL REPRESENTATION")
print("=" * 70)
print("\nBefore shallow copy:")
print("  lst1 ──────> [ ref_to_inner1, ref_to_inner2 ]")
print("                      ↓              ↓")
print("                 ['a', 'b']      ['c', 'd']")
print("")
print("After shallow copy:")
print("  lst1 ──────> [ ref_to_inner1, ref_to_inner2 ]  (address: 0x123)")
print("                      ↓              ↓")
print("  lst2 ──────> [ ref_to_inner1, ref_to_inner2 ]  (address: 0x456)")
print("                      ↓              ↓")
print("                 ['a', 'b']      ['c', 'd']")
print("")
print("  ⚠️  Different outer containers BUT same inner references!")
print("")
print("After lst1[0].append('x'):")
print("  lst1 ──────> [ ref_to_inner1, ref_to_inner2 ]")
print("                      ↓")
print("  lst2 ──────> [ ref_to_inner1, ref_to_inner2 ]")
print("                      ↓")
print("                 ['a', 'b', 'x']  ← Both point here!")
print("")
print("=" * 70)
print("SUMMARY")
print("=" * 70)
print("• Shallow copy creates NEW outer list (different address)")
print("• BUT copies only the REFERENCES to inner objects")
print("• Inner objects themselves are NOT copied (same addresses)")
print("• Modifying an inner object affects both lists")
print("• This is why it's called 'shallow' - only copies the surface!")

In [ ]:
# DEEP COPY vs SHALLOW COPY - The CRITICAL Difference for MUTABLE objects

import copy

lst1 = [['a', 'b'], ['c', 'd']]
lst2_shallow = copy.copy(lst1)
lst3_deep = copy.deepcopy(lst1)

print("=" * 70)
print("SHALLOW COPY vs DEEP COPY - MUTABLE INNER OBJECTS (lists)")
print("=" * 70)

print("\n1. OUTER LIST ADDRESSES (all different):")
print(f"   lst1:         {hex(id(lst1))}")
print(f"   lst2_shallow: {hex(id(lst2_shallow))}")
print(f"   lst3_deep:    {hex(id(lst3_deep))}")

print("\n2. INNER LIST ADDRESSES (THIS IS THE KEY!):")
print(f"   lst1[0]:         {hex(id(lst1[0]))} → {lst1[0]}")
print(f"   lst2_shallow[0]: {hex(id(lst2_shallow[0]))} → {lst2_shallow[0]}  ⚠️  SAME as lst1[0]!")
print(f"   lst3_deep[0]:    {hex(id(lst3_deep[0]))} → {lst3_deep[0]}  ✅ DIFFERENT!")

print(f"\n   lst1[1]:         {hex(id(lst1[1]))} → {lst1[1]}")
print(f"   lst2_shallow[1]: {hex(id(lst2_shallow[1]))} → {lst2_shallow[1]}  ⚠️  SAME as lst1[1]!")
print(f"   lst3_deep[1]:    {hex(id(lst3_deep[1]))} → {lst3_deep[1]}  ✅ DIFFERENT!")

print("\n" + "=" * 70)
print("TESTING: Modify lst1[0]")
print("=" * 70)

print(f"\nBefore modification:")
print(f"  lst1:         {lst1}")
print(f"  lst2_shallow: {lst2_shallow}")
print(f"  lst3_deep:    {lst3_deep}")

lst1[0].append('X')

print(f"\nAfter lst1[0].append('X'):")
print(f"  lst1:         {lst1}")
print(f"  lst2_shallow: {lst2_shallow}  ⚠️  CHANGED! (shares inner list)")
print(f"  lst3_deep:    {lst3_deep}  ✅ UNCHANGED! (independent inner list)")

print("\n" + "=" * 70)
print("WHAT ABOUT THE STRINGS INSIDE?")
print("=" * 70)

lst1_new = [['a', 'b'], ['c', 'd']]
lst2_deep = copy.deepcopy(lst1_new)

print(f"\nlst1_new[0][0]: '{lst1_new[0][0]}' address: {hex(id(lst1_new[0][0]))}")
print(f"lst2_deep[0][0]: '{lst2_deep[0][0]}' address: {hex(id(lst2_deep[0][0]))}")
print(f"Same? {lst1_new[0][0] is lst2_deep[0][0]} ← Yes, because strings are IMMUTABLE")

print("\n" + "=" * 70)
print("THE KEY DIFFERENCE")
print("=" * 70)
print("MUTABLE objects (lists, dicts, sets):")
print("  • Shallow copy: SHARES inner objects (same addresses)")
print("  • Deep copy:    CREATES NEW inner objects (different addresses)")
print("")
print("IMMUTABLE objects (str, int, float, tuple):")
print("  • Shallow copy: Shares (Python optimization)")
print("  • Deep copy:    ALSO shares (no need to copy, they can't change!)")
print("")
print("✅ Deep copy ONLY matters for nested MUTABLE structures!")

In [ ]:
# THE KEY: Modifying vs Replacing - Why shared addresses don't matter for immutables

import copy

print("=" * 70)
print("SIMPLE LIST: lst1 = [1, 2, 3]")
print("=" * 70)

lst1 = [1, 2, 3]
lst2 = copy.copy(lst1)  # Shallow copy
lst3 = copy.deepcopy(lst1)  # Deep copy

print(f"\nOuter lists:")
print(f"lst1: {hex(id(lst1))}")
print(f"lst2: {hex(id(lst2))} - Different from lst1")
print(f"lst3: {hex(id(lst3))} - Different from lst1")

print(f"\nInner objects (integers):")
print(f"lst1[0]: {lst1[0]} at {hex(id(lst1[0]))}")
print(f"lst2[0]: {lst2[0]} at {hex(id(lst2[0]))} - SAME address!")
print(f"lst3[0]: {lst3[0]} at {hex(id(lst3[0]))} - SAME address!")

print("\n" + "=" * 70)
print("CRITICAL: You CANNOT modify an integer!")
print("=" * 70)

print("\nIntegers are IMMUTABLE. You can't change the value of 1.")
print("When you do lst1[0] = 5, you're NOT modifying 1.")
print("You're REPLACING the reference with a new reference to 5.")

print(f"\nBefore: lst1 = {lst1}, lst2 = {lst2}, lst3 = {lst3}")
print("\nExecuting: lst1[0] = 999")
lst1[0] = 999

print(f"After:  lst1 = {lst1}")
print(f"        lst2 = {lst2} - Unchanged!")
print(f"        lst3 = {lst3} - Unchanged!")

print("\n" + "=" * 70)
print("CONTRAST WITH MUTABLE OBJECTS (nested lists)")
print("=" * 70)

lst1 = [['a', 'b'], ['c', 'd']]
lst2_shallow = copy.copy(lst1)
lst3_deep = copy.deepcopy(lst1)

print(f"\nInner list addresses:")
print(f"lst1[0]: {hex(id(lst1[0]))}")
print(f"lst2_shallow[0]: {hex(id(lst2_shallow[0]))} - SAME as lst1[0]")
print(f"lst3_deep[0]: {hex(id(lst3_deep[0]))} - DIFFERENT from lst1[0]")

print(f"\nBefore: lst1 = {lst1}, lst2_shallow = {lst2_shallow}, lst3_deep = {lst3_deep}")
print("\nExecuting: lst1[0].append('X')  ← MODIFYING the list object")

lst1[0].append('X')

print(f"\nAfter:  lst1 = {lst1}")
print(f"        lst2_shallow = {lst2_shallow} - CHANGED! (shared object)")
print(f"        lst3_deep = {lst3_deep} - Unchanged! (independent object)")

print("\n" + "=" * 70)
print("THE ANSWER")
print("=" * 70)
print("For lst1 = [1, 2, 3]:")
print("  • Lists have different addresses ✓")
print("  • Integers have SAME addresses ✓")
print("  • BUT integers are IMMUTABLE - you can't modify them!")
print("  • lst1[0] = 5 REPLACES the reference, doesn't modify the integer")
print("  • Since lists are independent, replacement only affects lst1")
print("")
print("For lst1 = [['a'], ['b']]:")
print("  • Shallow copy: Inner lists have SAME addresses")
print("  • Inner lists are MUTABLE - you CAN modify them!")
print("  • lst1[0].append('x') MODIFIES the shared list object")
print("  • Both lst1 and lst2 see the change")
print("")
print("For lst1 = [['a'], ['b']] with deep copy:")
print("  • Deep copy: Inner lists have DIFFERENT addresses")
print("  • lst1[0].append('x') MODIFIES only lst1's inner list")
print("  • lst3 is unaffected because it has its own inner list")
print("")
print("✅ Shared addresses only matter if the object is MUTABLE!")

In [50]:
class CustomLogger:
    def __init__(self, name):
        print(self.__class__.__name__)
        self.logger = logging.getLogger(name)
        self.logger.setLevel(logging.INFO)

    def add_stream_handler(self):
        self.logger.addHandler(logging.StreamHandler())

    def add_file_handler(self, filename):
        self.logger.addHandler(logging.FileHandler(filename))

    def add_null_handler(self):
        self.logger.addHandler(logging.NullHandler())

    def remove_all_handlers(self):
        for handler in self.logger.handlers[:]:
            self.logger.removeHandler(handler)

In [51]:
log = CustomLogger('MyAppLogger')

CustomLogger
